In [ ]:
import os

# Desinstala numpy y cualquier paquete conflictivo
!pip uninstall -y numpy torch torchvision torchaudio torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv -q

# Instala numpy compatible (ANTES de que torch lo cargue)
!pip install numpy==1.24.4

# Mensaje al usuario
print("NumPy 1.24.4 instalado correctamente.")
print("Reiniciando entorno para que los cambios surtan efecto...")

# Reinicia el entorno de ejecución automáticamente (muy importante)
os.kill(os.getpid(), 9)


In [ ]:

# Instalación de PyTorch con CUDA 11.8
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

# Instalación de dependencias específicas de PyTorch Geometric
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html

# Instalación de torch-geometric y librerías útiles
!pip install torch-geometric matplotlib pandas optuna optuna-dashboard

!pip install pyngrok --quiet


In [ ]:
# Eliminar carpeta antigua si existe, para evitar que se quede en  el caché
!rm -rf ia_thermal

# Clonar versión actualizada desde GitHub
!git clone --branch Ernesto https://github.com/dgbarcena/ia_thermal.git

%cd ia_thermal/ernestoemedina/NNConv_MK1

In [ ]:
# Añadir ruta al sistema para importar módulos locales
import sys
sys.path.append('/content/ia_thermal/ernestoemedina/NNConv_MK1')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Optuna

In [ ]:
import optuna
import optuna_dashboard
import torch
import os
import threading
from torch.optim.lr_scheduler import ReduceLROnPlateau
from optuna_dashboard import run_server
from pyngrok import ngrok
from nnconv_model import NNConvNet
from train_eval import train, evaluate
from dataset_utils import get_dataloaders_optuna

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    dir_path = "/content/drive/MyDrive/ErnestoData"
    db_path = "/content/drive/MyDrive/ErnestoData/Optuna/nnconv_optuna_1.db"
else:
    dir_path = os.getcwd()
    db_path = os.path.join(dir_path, "nnconv_optuna_1.db")

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)


In [ ]:

def get_dataloaders(batch_size, dir_path):
    return get_dataloaders_optuna(batch_size, dir_path)


In [ ]:
def objective(trial):
    # --- Hiperparámetros a optimizar ---
    num_layers = trial.suggest_int("num_layers", 1, 10)
    hidden_dim = trial.suggest_categorical("hidden_dim", [8, 9, 16, 32, 64, 128])
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.2)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])

    # --- Cargar datos ---
    train_loader, val_loader, test_loader, input_dim, norm_info = get_dataloaders(batch_size=batch_size, dir_path=dir_path)


    # --- Crear modelo ---
    model = NNConvNet(
        input_dim=input_dim,
        edge_dim=3,
        hidden_dim=hidden_dim,
        output_dim=1,
        num_layers=num_layers,
        use_dropout=True,
        dropout_rate=dropout_rate,
        use_batchnorm=True,
        use_residual=True
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # --- Scheduler con ReduceLROnPlateau ---
    scheduler = ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=10,
        min_lr=1e-6
    )

    # --- Entrenamiento + Evaluación en validación ---
    for epoch in range(50):
        train(
            model,
            train_loader,
            optimizer,
            device,
            norm_info=norm_info,
            use_physics=True,
            lambda_physics=0.0001,
            use_boundary_loss=True,
            lambda_boundary=1,
            use_heater_loss=True,
            lambda_heater=10
        )

        val_metrics = evaluate(
            model,
            val_loader,
            device,
            norm_info=norm_info,
            error_threshold=1.0,
            use_physics=True,
            use_boundary_loss=True,
            use_heater_loss=True,
            lambda_physics=0.003,
            lambda_boundary=1,
            lambda_heater=10
        )

        val_total_loss = val_metrics[-1]
        scheduler.step(val_total_loss)

    return val_total_loss  # Métrica a minimizar

## Estudio

In [ ]:
""" # Ruta al archivo .db

port = 8080

# Abrir túnel ngrok
public_url = ngrok.connect(port)
print(f"Abre este enlace para ver Optuna Dashboard:\n{public_url}")

# Ejecutar el dashboard en segundo plano con threading
thread = threading.Thread(
    target=run_server,
    args=(f"sqlite:///{db_path}",),
    kwargs={"port": port, "host": "0.0.0.0", "no_browser": True},
    daemon=True
)
thread.start() """

In [ ]:
# Crear estudio y ejecutar optimización
study = optuna.create_study(
    direction="minimize",
    study_name="nnconv_study_1",
    storage=f"sqlite:///db_path",
    load_if_exists=True
)

study.optimize(objective, n_trials=100)

In [ ]:

print("Mejor conjunto de hiperparámetros encontrados:")
for key, val in study.best_trial.params.items():
    print(f"{key}: {val}")


In [ ]:

import optuna.visualization.matplotlib as vis

fig1 = vis.plot_optimization_history(study)
fig1.show()

fig2 = vis.plot_param_importances(study)
fig2.show()
